https://python.langchain.com/v0.1/docs/use_cases/sql/quickstart/

# DB Part

SQLite 是一个轻量级的数据库，设计目的是为了嵌入到应用程序中。它不需要一个独立的服务器进程或操作系统的安装和配置，数据库就是一个单一的磁盘文件。比 MySQL 要来得更轻量。

1. `apt install sqlite3`
2. `wget https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql`
3. `sqlite3 Chinook.db` 进入 sqlite 的命令行
   1. `.read Chinook_Sqlite.sql`
   2. `SELECT * FROM Artist LIMIT 10;`
   3. `.quit`

# Langchian Part

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [4]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(base_url="https://api.deepseek.com" , model="deepseek-chat", temperature=0)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})
response

'SQLQuery: SELECT COUNT(*) AS EmployeeCount FROM "Employee"\nSQLResult:'

In [7]:
db.run('SELECT COUNT(*) AS EmployeeCount FROM "Employee"')

'[(8,)]'

In [8]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [11]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "How many employees are there"})

'Error: (sqlite3.OperationalError) near "SQLQuery": syntax error\n[SQL: SQLQuery: SELECT COUNT(*) FROM "Employee"\nSQLResult:]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)'

In [12]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})

' The provided SQL query contains a syntax error due to the inclusion of the word "SQLQuery" in the query itself. The correct SQL query to count the number of employees should be:\n\n```sql\nSELECT COUNT(*) FROM "Employee"\n```\n\nHowever, since the SQL result indicates an error and no actual count is provided, I cannot give you the number of employees based on the information given. Please execute the corrected SQL query and provide the result to get the answer to the question.'